In [1]:
import pandas as pd
import numpy as np
import hvplot
import matplotlib
import statsmodels.api as sm
import math

In [8]:
df_import = pd.read_csv('aussie_equities_scoring_python_15072024.csv', index_col=0, encoding='latin-1')
data1 = df_import.apply(lambda x: pd.to_numeric(x, errors='coerce'))
data2 = data1.copy()
data2.columns = [data1.columns, df_import.iloc[0]]

data_cleaned = data2.iloc[1:]

df_export = pd.read_csv('new_scoring.csv')

In [ ]:
classification = df_export[['Unnamed: 0', 'apir_ticker', 'Classification']]
classification.index = classification['apir_ticker']
classification

In [142]:
#beta estimation
output = {}
output2 = {}

data_drop_level_0 = data_cleaned.droplevel(axis=1,level=0)

for i in data_drop_level_0.iloc[:,5:].columns:
    y = data_drop_level_0.loc[:,i].iloc[-120:]
    
    if classification.loc[i]['Classification'] == 'Beta' or classification.loc[i]['Classification'] == 'Alpha':
        x = data_drop_level_0.loc[:,'Market'].iloc[-120:]

    else: 
        classification_value = classification.loc[i]['Classification']
        if classification_value == 'Growth':
            columns_to_select = ['Market', 'Value']
            x = data_drop_level_0.loc[:, columns_to_select].iloc[-120:]
            x['Value'] = -x['Value']
            x = x.rename(columns={'Value':'Growth'})

        else:
            columns_to_select = ['Market', classification_value]
            x = data_drop_level_0.loc[:, columns_to_select].iloc[-120:]

    if y.count() < 24: 
            output[i] = np.nan
            output2[i] = np.nan
        
    else: 
        model = sm.OLS(y, sm.add_constant(x), missing='drop').fit()
        output[i] = model.params
        output2[i] = model.rsquared

coefficients = pd.DataFrame(output)
coefficients.columns = data2.iloc[:,5:].columns
coefficients = coefficients.T
coefs = coefficients.reset_index()
coefs.index = coefs['apir_ticker']
coefs['classification'] = classification['Classification']

In [151]:
coefs

,level_0,apir_ticker,Growth,Market,Momentum,Quality,Size,Value,const,classification
apir_ticker,,,,,,,,,,
TYN0038AU,Tyndall Australian Share Income,TYN0038AU,NaN,0.981089,NaN,NaN,NaN,0.151366,-0.001578,Value
BFL0002AU,Bennelong Concentrated Australian Eq,BFL0002AU,NaN,0.928329,NaN,NaN,0.534256,NaN,0.001251,Size
BFL0001AU,Bennelong Australian Equities,BFL0001AU,0.366751,1.072465,NaN,NaN,NaN,NaN,-0.000444,Growth
WHT0009AU,Solaris Core Australian Equity Inst,WHT0009AU,0.016514,1.031125,NaN,NaN,NaN,NaN,-0.000605,Growth
ECL0005AU,Ellerston Australian Share Fund,ECL0005AU,NaN,0.928441,NaN,NaN,0.326919,NaN,-0.001251,Size
...,...,...,...,...,...,...,...,...,...,...
IMLC,IML Concntr AUS Shr Fd (Quoted Mngd Fd),IMLC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alpha
DACE,Dimensional AustralianCoreEquityTrustETF,DACE,NaN,1.020580,NaN,NaN,0.189966,NaN,-0.000862,Size
G200,BetasharesWthBdrAUS200Grd30-40%LVRCpxETF,G200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alpha
